##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFRecords と tf.train.Example

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

TFRecord 形式は一連のバイナリレコードを格納するための単純な形式です。

[プロトコルバッファ](https://developers.google.com/protocol-buffers/)は、構造化データを効率的にシリアル化するクロスプラットフォームのクロス言語ライブラリです。

Protocol messages are defined by `.proto` files, these are often the easiest way to understand a message type.

`tf.train.Example` メッセージ（または protobuf）は柔軟なメッセージ型で、`{"string": value}` マッピングを表現します。TensorFlow と使用するように設計されており、[TFX](https://www.tensorflow.org/tfx/) などのより高位な API で使用されます。

This notebook demonstrates how to create, parse, and use the `tf.train.Example` message, and then serialize, write, and read `tf.train.Example` messages to and from `.tfrecord` files.

Note: While useful, these structures are optional. There is no need to convert existing code to use TFRecords, unless you are [using tf.data](https://www.tensorflow.org/guide/data) and reading data is still the bottleneck to training. You can refer to [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance) for dataset performance tips.

Note: In general, you should shard your data across multiple files so that you can parallelize I/O (within a single host or across multiple hosts). The rule of thumb is to have at least 10 times as many files as there will be hosts reading data. At the same time, each file should be large enough (at least 10 MB+ and ideally 100 MB+) so that you can benefit from I/O prefetching. For example, say you have `X` GB of data and you plan to train on up to `N` hosts. Ideally, you should shard the data to ~`10*N` files, as long as ~`X/(10*N)` is 10 MB+ (and ideally 100 MB+). If it is less than that, you might need to create fewer shards to trade off parallelism benefits and I/O prefetching benefits.

## セットアップ

In [ ]:
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.train.Example`

### `tf.train.Example` のデータ型

基本的に、`tf.train.Example` は `{"string": tf.train.Feature}` というマッピングです。

`tf.train.Feature` メッセージ型は次の 3 つの型のうち 1 つを取ることができます（[`.proto` ファイル](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto)を参照）。ほとんどのその他の汎用的なデータ型は、強制的に次のいずれかにすることができます。

1. `tf.train.BytesList` (the following types can be coerced)

- `string`
- `byte`

1. `tf.train.FloatList` (the following types can be coerced)

- `float` (`float32`)
- `double` (`float64`)

1. `tf.train.Int64List` (the following types can be coerced)

- `bool`
- `enum`
- `int32`
- `uint32`
- `int64`
- `uint64`

In order to convert a standard TensorFlow type to a `tf.train.Example`-compatible `tf.train.Feature`, you can use the shortcut functions below. Note that each function takes a scalar input value and returns a `tf.train.Feature` containing one of the three `list` types above:

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

Note: To stay simple, this example only uses scalar inputs. The simplest way to handle non-scalar features is to use `tf.io.serialize_tensor` to convert tensors to binary-strings. Strings are scalars in TensorFlow. Use `tf.io.parse_tensor` to convert the binary-string back to a tensor.

Below are some examples of how these functions work. Note the varying input types and the standardized output types. If the input type for a function does not match one of the coercible types stated above, the function will raise an exception (e.g. `_int64_feature(1.0)` will error out because `1.0` is a float—therefore, it should be used with the `_float_feature` function instead):

In [ ]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

proto メッセージはすべて `.SerializeToString` メソッドを使ってバイナリ文字列にシリアル化できます。

In [ ]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### `tf.train.Example` メッセージを作成する

既存のデータから `tf.train.Example` を作成するとします。実際には、データセットの出処はどこでもよいのですが、1 件の観測記録から `tf.train.Example` メッセージを作成する手順は変わりません。

1. 各観測記録において、上記のいずれかの関数を使用して、それぞれの値を 3 つの互換性のある型のいずれかを含む `tf.train.Feature` に変換する必要があります。

2. 特徴量名の文字列から、手順 1 で生成されたエンコード済みの特徴量の値にマップ（ディクショナリ）を作成します。

3. 手順 2 で生成されたマップを [`Features` メッセージ](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85)に変換します。

このノートブックでは、NumPy を使ってデータセットを作成します。

このデータセットには次の 4 つの特徴量があります。

- `False` または `True` を表す同等の発生確率を持つブール型の特徴量。
- `[0, 5]` から一様にランダムに選択される整数の特徴量。
- 整数の特徴量をインデックスとして使用することによって文字列のテーブルから生成される文字列の特徴量。
- 標準正規分布の浮動小数点数の特徴量。

サンプルは上記の分布から独立して同じ様に分布した 10,000 件の観測記録からなるものとします。

In [ ]:
# データセットに含まれる観測結果の件数
n_observations = int(1e4)

# ブール特徴量 False または True としてエンコードされている
feature0 = np.random.choice([False, True], n_observations)

# 整数特徴量  -10000 から 10000 の間の乱数
feature1 = np.random.randint(0, 5, n_observations)

# バイト特徴量
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# 浮動小数点数特徴量 標準正規分布から発生
feature3 = np.random.randn(n_observations)

これらの特徴量は、`_bytes_feature`、`_float_feature`、`_int64_feature` のいずれかを使って `tf.train.Example` と互換性のある型に強制されます。その後で、エンコード済みの特徴量から `tf.train.Example` メッセージを作成できます。

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

たとえば、データセットに `[False, 4, bytes('goat'), 0.9876]` という 1 つの観測記録があるとします。`create_message()` を使うとこの観測記録から `tf.train.Example` メッセージを作成して出力できます。それぞれの観測記録は上記に従って `Features` メッセージとして書き込まれます。`tf.train.Example` [メッセージ](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88)は、この `Features` メッセージを囲むラッパーに過ぎないことに注意してください。

In [ ]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

メッセージをデコードするには、`tf.train.Example.FromString` メソッドを使用します。

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

## TFRecords format details

A TFRecord file contains a sequence of records. The file can only be read sequentially.

Each record contains a byte-string, for the data-payload, plus the data-length, and  CRC-32C ([32-bit CRC](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#CRC-32_algorithm) using the [Castagnoli polynomial](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#Standards_and_common_use)) hashes for integrity checking.

Each record is stored in the following formats:

    uint64 length
    uint32 masked_crc32_of_length
    byte   data[length]
    uint32 masked_crc32_of_data

The records are concatenated together to produce the file. CRCs are
[described here](https://en.wikipedia.org/wiki/Cyclic_redundancy_check), and
the mask of a CRC is:

    masked_crc = ((crc >> 15) | (crc << 17)) + 0xa282ead8ul


Note: There is no requirement to use `tf.train.Example` in TFRecord files. `tf.train.Example` is just a method of serializing dictionaries to byte-strings. Any byte-string that can be decoded in TensorFlow could be stored in a TFRecord file. Examples include: lines of text, JSON (using `tf.io.decode_json_example`), encoded image data, or serialized `tf.Tensors` (using `tf.io.serialize_tensor`/`tf.io.parse_tensor`). See the `tf.io` module for more options.

## `tf.data` を使った TFRecord ファイル

`tf.data` モジュールには、TensorFlow でデータを読み書きするツールも含まれています。

### TFRecord ファイルを書き込む

データをデータセットに書き込む最も簡単な方法は `from_tensor_slices` メソッドを使用する方法です。

配列に適用すると、このメソッドはスカラー値のデータセットを返します。

In [ ]:
tf.data.Dataset.from_tensor_slices(feature1)

配列のタプルに適用すると、タプルのデータセットを返します。

In [ ]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

In [ ]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)

`Dataset` のそれぞれの要素に関数を適用するには、`tf.data.Dataset.map` メソッドを使用します。

マップされる関数は TensorFlow のグラフモードで動作する必要があり、`tf.Tensors` を処理して返す必要があります。`serialize_example` のような非テンソル関数は、互換性を得るために `tf.py_function` で囲むことができます。

`tf.py_function` を使用する際は、形状と型を指定する必要があります。指定しない場合、形状と型を利用できません。

In [ ]:
def tf_serialize_example(f0,f1,f2,f3):
  tf_string = tf.py_function(
    serialize_example, 
    (f0,f1,f2,f3),  # 上記の関数にこれらの引数を渡す
    tf.string)      # 戻り値の型は tf.string
  return tf.reshape(tf_string, ()) # 結果はスカラー

In [ ]:
tf_serialize_example(f0, f1, f2, f3)

この関数をデータセットのそれぞれの要素に適用します。

In [ ]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

In [ ]:
def generator():
  for features in features_dataset:
    yield serialize_example(*features)

In [ ]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [ ]:
serialized_features_dataset

TFRecord ファイルに書き込みます。

In [ ]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

### TFRecord ファイルを読み取る

`tf.data.TFRecordDataset` クラスを使って TFRecord ファイルを読み取ることもできます。

More information on consuming TFRecord files using `tf.data` can be found in the [tf.data: Build TensorFlow input pipelines](https://www.tensorflow.org/guide/data#consuming_tfrecord_data) guide.

`TFRecordDataset` を使うと、入力データを標準化し、パフォーマンスを最適化するのに役立ちます。

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

この時点で、データセットにはシリアル化された `tf.train.Example` メッセージが含まれています。データセットをイテレートすると、これらのメッセージはスカラーの文字列テンソルとして返されます。

`.take` メソッドを使って最初の 10 件のレコードのみを表示します。

注意: `tf.data.Dataset` をイテレートできるのは、Eager execution が有効になっている場合のみです。

In [ ]:
for raw_record in raw_dataset.take(10):
    print(repr(raw_record))

These tensors can be parsed using the function below. Note that the `feature_description` is necessary here because `tf.data.Dataset`s use graph-execution, and need this description to build their shape and type signature:

In [ ]:
# Create a description of the features.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

または、`tf.parse example` を使ってバッチ全体を一度に解析します。`tf.data.Dataset.map` メソッドを使って、データセットの各項目にこの関数を適用します。

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset 

Eager Execution を使ってデータセット中の観測記録を表示します。このデータセットには 10,000 件の観測記録がありますが、最初の 10 個だけ表示します。
 データは特徴量のディクショナリの形で表示されます。それぞれの項目は `tf.Tensor` であり、このテンソルの `numpy` 要素は特徴量を表します。

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(raw_record))

Here, the `tf.parse_example` function unpacks the `tf.train.Example` fields into standard tensors.

## Python の TFRecord ファイル

`tf.io` モジュールには、TFRecord ファイルを読み書きするための純粋な Python 関数も含まれています。

### TFRecord ファイルを書き込む

Next, write the 10,000 observations to the file `test.tfrecord`. Each observation is converted to a `tf.train.Example` message, then written to file. You can then verify that the file `test.tfrecord` has been created:

In [ ]:
# Write the `tf.train.Example` observations to the file.
with tf.io.TFRecordWriter(filename) as writer:
  for i in range(n_observations):
    example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
    writer.write(example)

In [ ]:
!du -sh {filename}

### Reading a TFRecord file

These serialized tensors can be easily parsed using `tf.train.Example.ParseFromString`:

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

That returns a `tf.train.Example` proto which is dificult to use as is, but it's fundamentally a representation of a:

```
Dict[str,
     Union[List[float],
           List[int],
           List[str]]]
```

The following code manually converts the `Example` to a dictionary of NumPy arrays, without using TensorFlow Ops. Refer to [the PROTO file](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) for detials.

In [ ]:
result = {}
# example.features.feature is the dictionary
for key, feature in example.features.feature.items():
  # The values are the Feature objects which contain a `kind` which contains:
  # one of three fields: bytes_list, float_list, int64_list

  kind = feature.WhichOneof('kind')
  result[key] = np.array(getattr(feature, kind).value)

result

## ウォークスルー: 画像データの読み書き

これは、TFRecord を使って画像データを読み書きする方法を示すエンドツーエンドの例です。画像を入力データとして使用し、そのデータを TFRecord ファイルとして書き込んでから、ファイルを読み取り直して画像を表示します。

これは、同じ入力データセットに複数のモデルを使用する場合などに役立ちます。画像データを未加工のまま保存する代わりに、事前に TFRecord 形式に処理しておくことが可能で、その形式を以降の処理やモデル構築に使用することができます。

まずは、雪の中の猫の[画像](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg)と、ニューヨーク市にある建設中のウイリアムズバーグ橋の[写真](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg)をダウンロードしましょう。

### 画像をフェッチする

In [ ]:
cat_in_snow  = tf.keras.utils.get_file('320px-Felis_catus-cat_on_snow.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg')
williamsburg_bridge = tf.keras.utils.get_file('194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [ ]:
display.display(display.Image(filename=cat_in_snow))
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
display.display(display.Image(filename=williamsburg_bridge))
display.display(display.HTML('<a "href=https://commons.wikimedia.org/wiki/File:New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg">From Wikimedia</a>'))

### TFRecord ファイルを書き込む

前と同じように、この特徴量を `tf.train.Example` と互換性のあるデータ型にエンコードします。この場合には、生の画像文字列の特徴量だけではなく、高さ、幅、深度、および任意の `label` 特徴量も保存します。ラベルはファイルに書き込む際に猫の画像と橋の画像を区別するために使用されます。猫の画像には `0` を、橋の画像には `1` を使用します。

In [ ]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [ ]:
# 猫の画像を使った例
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# 関連する特徴量のディクショナリを作成
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

すべての特徴量が `tf.train.Example` メッセージに保存されたのがわかります。上記のコードを関数化し、このサンプルメッセージを `images.tfrecords` というファイルに書き込みます。

In [ ]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [ ]:
!du -sh {record_file}

### TFRecord ファイルを読み取る

You now have the file—`images.tfrecords`—and can now iterate over the records in it to read back what you wrote. Given that in this example you will only reproduce the image, the only feature you will need is the raw image string. Extract it using the getters described above, namely `example.features.feature['image_raw'].bytes_list.value[0]`. You can also use the labels to determine which record is the cat and which one is the bridge:

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

TFRecord ファイルから画像を復元します。

In [ ]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))